In [3]:
# 뉴스 lda 분석
import re
import os
import pandas as pd
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer

# 기본 경로 설정
base_path = r"C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01"
stopwords_path = os.path.join(base_path, "stopwords_news.txt")

# 불용어 파일 읽기 함수
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stop_words = set(line.strip() for line in f)
        return stop_words
    except FileNotFoundError:
        print(f"불용어 파일이 존재하지 않습니다: {file_path}")
        return set()

# 텍스트 전처리 함수 (간단한 정규식 기반 토크나이저 사용)
def preprocess_text(text, stopwords_path):
    stop_words = load_stopwords(stopwords_path)  # 최신 불용어 목록 로드
    tokenizer = RegexpTokenizer(r'\w+')  # 단어 기준 토크나이저
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    tokens = tokenizer.tokenize(text)  # 텍스트를 단어 단위로 토크나이징
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return tokens

# LDA 분석 함수
def lda_analysis(file_name, stopwords_path, num_topics=5, passes=10):
    try:
        # 파일 경로 설정
        file_path = os.path.join(base_path, file_name)
        
        # 데이터 로드
        data = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', engine='python')
        print(f"파일 로드 성공: {file_name}")

        # 데이터 컬럼 이름을 소문자로 변환
        data.columns = data.columns.str.lower()

        # 'description' 컬럼 전처리
        data['description'] = data['description'].fillna('')  # 결측값 처리
        data['processed'] = data['description'].apply(lambda text: preprocess_text(text, stopwords_path))

        # 말뭉치 및 사전 생성
        dictionary = corpora.Dictionary(data['processed'])
        corpus = [dictionary.doc2bow(text) for text in data['processed']]

        # LDA 모델 학습
        lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

        # 토픽 출력
        print(f"\n=== {file_name}의 토픽 ===")
        topics = lda_model.print_topics(num_words=5)
        for idx, topic in topics:
            print(f"토픽 {idx}: {topic}")
    except FileNotFoundError as fnf_error:
        print(f"파일을 찾을 수 없습니다: {fnf_error}")
    except Exception as e:
        print(f"{file_name} 처리 중 오류 발생: {e}")

# 5개년 데이터 처리
file_names = [f"articles_20대_소비_{year}.csv" for year in range(2020, 2025)]
for file_name in file_names:
    lda_analysis(file_name, stopwords_path)

파일 로드 성공: articles_20대_소비_2020.csv

=== articles_20대_소비_2020.csv의 토픽 ===
토픽 0: 0.002*"코로나" + 0.001*"상하이지부장은" + 0.001*"무역협회" + 0.001*"컬러를" + 0.001*"덕후"
토픽 1: 0.004*"코로나" + 0.001*"개정" + 0.001*"감소했지만" + 0.001*"MZ세대" + 0.001*"청량하고"
토픽 2: 0.003*"코로나" + 0.001*"스트리밍" + 0.001*"원장은" + 0.001*"공식" + 0.001*"강한"
토픽 3: 0.006*"코로나" + 0.002*"SNS" + 0.002*"트위터" + 0.001*"감소했다" + 0.001*"건이었으며"
토픽 4: 0.004*"코로나" + 0.002*"경험이" + 0.001*"원장" + 0.001*"게임으로" + 0.001*"만든"
파일 로드 성공: articles_20대_소비_2021.csv

=== articles_20대_소비_2021.csv의 토픽 ===
토픽 0: 0.004*"코로나" + 0.001*"지출은" + 0.001*"사회적으로" + 0.001*"의료비" + 0.001*"역대"
토픽 1: 0.003*"코로나" + 0.002*"MZ세대" + 0.001*"MZ세대를" + 0.001*"MZ세대의" + 0.001*"영역에"
토픽 2: 0.002*"코로나" + 0.002*"MZ세대" + 0.002*"히키코모리" + 0.001*"관련해" + 0.001*"선거와"
토픽 3: 0.002*"코로나" + 0.001*"활동" + 0.001*"경험" + 0.001*"MZ세대" + 0.001*"분야에"
토픽 4: 0.004*"코로나" + 0.002*"경험이" + 0.001*"대표적인" + 0.001*"MZ세대가" + 0.001*"MZ세대"
파일 로드 성공: articles_20대_소비_2022.csv

=== articles_20대_소비_2022.csv의 토픽 ===
토픽 0: 0.001*"MZ세대가" +

In [44]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [10]:
# 뉴스 wordcloud
import re
import os
import pandas as pd
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict
import chardet
from matplotlib import font_manager, rc

# 기본 경로 설정
base_path = r"C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01"
stopwords_path = os.path.join(base_path, "stopwords_news.txt")

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows의 경우
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 불용어 파일 읽기 함수
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stop_words = set(line.strip() for line in f)
        return stop_words
    except FileNotFoundError:
        print(f"불용어 파일이 존재하지 않습니다: {file_path}")
        return set()

# 텍스트 전처리 함수 (간단한 정규식 기반 토크나이저 사용)
def preprocess_text(text, stopwords_path):
    stop_words = load_stopwords(stopwords_path)  # 최신 불용어 목록 로드
    tokenizer = RegexpTokenizer(r'\w+')  # 단어 기준 토크나이저
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    tokens = tokenizer.tokenize(text)  # 텍스트를 단어 단위로 토크나이징
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return tokens

# 워드 클라우드 생성 함수
def generate_wordcloud(title, texts):
    try:
        # 모든 텍스트를 하나로 합침
        all_text = ' '.join(texts)
        # 전처리
        tokens = preprocess_text(all_text, stopwords_path)
        # 단어 빈도 계산
        freq = defaultdict(int)
        for word in tokens:
            freq[word] += 1
        if not freq:
            print(f"{title} 워드 클라우드에 사용할 단어가 없습니다.")
            return
        # 워드 클라우드 생성
        wordcloud = WordCloud(
            font_path=font_path,  # 한글 폰트 경로 설정
            background_color='white',
            width=800,
            height=600
        ).generate_from_frequencies(freq)
        # 워드 클라우드 시각화
        plt.figure(figsize=(10, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(title, fontsize=20)
        plt.tight_layout(pad=0)
        # 워드 클라우드를 이미지 파일로 저장
        save_path = os.path.join(base_path, f"{title}.png")
        plt.savefig(save_path)
        plt.close()
        print(f"{title} 워드 클라우드 저장 완료: {save_path}")
    except Exception as e:
        print(f"{title} 워드 클라우드 생성 중 오류 발생: {e}")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 데이터 로드 및 그룹화
def load_and_group_data(file_names, group1_years, group2_years):
    group1_data = []
    group2_data = []
    print("\n=== 데이터 로드 시작 ===")
    for file_name in file_names:
        file_path = os.path.join(base_path, file_name)
        print(f"\n파일 로드: {file_path}")
        encoding = detect_encoding(file_path)
        data = pd.read_csv(file_path, encoding=encoding, on_bad_lines='skip', engine='python')
        if data is not None:
            # 데이터 컬럼 이름을 소문자로 변환 (문자열만)
            data.columns = [col.lower() if isinstance(col, str) else col for col in data.columns]
            # 텍스트 컬럼 자동 탐색
            text_columns = data.select_dtypes(include=['object']).columns.tolist()
            if not text_columns:
                print(f"문자열 데이터가 있는 컬럼이 {file_name}에 존재하지 않습니다.")
                continue
            # 우선 'description' 컬럼이 있으면 사용, 없으면 첫 번째 문자열 컬럼 사용
            if 'description' in text_columns:
                text_col = 'description'
            else:
                text_col = text_columns[0]
                print(f"'description' 컬럼이 {file_name}에 존재하지 않습니다. 대신 '{text_col}' 컬럼을 사용합니다.")
            # 선택된 컬럼의 데이터를 문자열로 변환
            data[text_col] = data[text_col].apply(lambda x: str(x) if not isinstance(x, str) else x).fillna('')
            # 문서 추출
            documents = data[text_col].tolist()
            if not documents:
                print(f"{file_name}에서 문서를 추출하지 못했습니다.")
                continue
            # 그룹에 따라 데이터 추가
            year_num = re.findall(r'\d{4}', file_name)[0]
            if year_num in group1_years:
                group1_data.extend(documents)
                print(f"{file_name} 데이터를 그룹1에 추가했습니다.")
            elif year_num in group2_years:
                group2_data.extend(documents)
                print(f"{file_name} 데이터를 그룹2에 추가했습니다.")
            else:
                print(f"{file_name}은 정의된 그룹에 속하지 않습니다.")
        else:
            print(f"{file_name} 데이터 로드에 실패했습니다.")
    # 데이터 확인
    print("\n=== 데이터 로드 완료 ===")
    print(f"그룹1 (2020-2021년) 데이터 수: {len(group1_data)}")
    print(f"그룹2 (2022-2024년) 데이터 수: {len(group2_data)}")
    return group1_data, group2_data

# 5개년 데이터 처리
file_names = [f"articles_20대_소비_{year}.csv" for year in range(2020, 2025)]
group1_years = ['2020', '2021']
group2_years = ['2022', '2023', '2024']

group1_data, group2_data = load_and_group_data(file_names, group1_years, group2_years)

# 워드 클라우드 생성 실행
if group1_data:
    generate_wordcloud("2020-2021년", group1_data)
else:
    print("그룹1에 데이터가 없습니다.")
if group2_data:
    generate_wordcloud("2022-2024년", group2_data)
else:
    print("그룹2에 데이터가 없습니다.")


=== 데이터 로드 시작 ===

파일 로드: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\articles_20대_소비_2020.csv
articles_20대_소비_2020.csv 데이터를 그룹1에 추가했습니다.

파일 로드: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\articles_20대_소비_2021.csv
articles_20대_소비_2021.csv 데이터를 그룹1에 추가했습니다.

파일 로드: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\articles_20대_소비_2022.csv
articles_20대_소비_2022.csv 데이터를 그룹2에 추가했습니다.

파일 로드: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\articles_20대_소비_2023.csv
articles_20대_소비_2023.csv 데이터를 그룹2에 추가했습니다.

파일 로드: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\articles_20대_소비_2024.csv
articles_20대_소비_2024.csv 데이터를 그룹2에 추가했습니다.

=== 데이터 로드 완료 ===
그룹1 (2020-2021년) 데이터 수: 1400
그룹2 (2022-2024년) 데이터 수: 2128
2020-2021년 워드 클라우드 저장 완료: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\2020-2021년.png
2022-2024년 워드 클라우드 저장 완료: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\2022-2024년.png


In [2]:
# 블로그 lda 분석
import re
import os
import json
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer

# 기본 경로 설정
base_path = r"C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01"
stopwords_path = os.path.join(base_path, "stopwords_blog.txt")

# 불용어 파일 읽기 함수
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stop_words = set(line.strip() for line in f)
        # print(f"불용어 목록 ({len(stop_words)}개): {list(stop_words)[:10]}...")  # 불용어 목록 일부 출력
        return stop_words
    except FileNotFoundError:
        print(f"불용어 파일이 존재하지 않습니다: {file_path}")
        return set()

# 텍스트 전처리 함수 (간단한 정규식 기반 토크나이저 사용)
def preprocess_text(text, stop_words):
    tokenizer = RegexpTokenizer(r'\w+')  # 단어 기준 토크나이저
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    tokens = tokenizer.tokenize(text)  # 텍스트를 단어 단위로 토크나이징
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    # print(f"전처리된 텍스트: {tokens[:10]}...")  # 전처리된 텍스트 일부 출력
    return tokens

# LDA 분석 함수
def lda_analysis(file_name, stopwords_path, num_topics=5, passes=10):
    try:
        # 파일 경로 설정
        file_path = os.path.join(base_path, file_name)
        
        # 데이터 로드
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 불용어 목록 로드
        stop_words = load_stopwords(stopwords_path)
        
        # 데이터 전처리
        all_texts = []
        for year_data in data:
            for entry in year_data.get(file_name.split('_')[2].split('.')[0], []):  # 연도 추출하여 사용
                content = entry.get('content', '')
                processed_text = preprocess_text(content, stop_words)
                if len(processed_text) > 5:  # 전처리된 텍스트가 일정 길이 이상인 경우에만 추가
                    all_texts.append(processed_text)
        
        if not all_texts:
            print(f"{file_name}에 유효한 텍스트가 없습니다.")
            return
        
        # 말뭉치 및 사전 생성
        dictionary = corpora.Dictionary(all_texts)
        corpus = [dictionary.doc2bow(text) for text in all_texts]

        # LDA 모델 학습
        lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

        # 토픽 출력
        print(f"\n=== {file_name}의 토픽 ===")
        topics = lda_model.print_topics(num_words=5)
        for idx, topic in topics:
            print(f"토픽 {idx}: {topic}")
    except FileNotFoundError as fnf_error:
        print(f"파일을 찾을 수 없습니다: {fnf_error}")
    except Exception as e:
        print(f"{file_name} 처리 중 오류 발생: {e}")

# JSON 파일 목록
file_names = [
    "naver_blog_2020.json",
    "naver_blog_2021.json",
    "naver_blog_2022.json",
    "naver_blog_2023.json",
    "naver_blog_2024.json"
]

# 각 파일에 대해 LDA 분석 수행
for file_name in file_names:
    lda_analysis(file_name, stopwords_path)


=== naver_blog_2020.json의 토픽 ===
토픽 0: 0.001*"직접" + 0.001*"보니" + 0.001*"사람이" + 0.001*"줄" + 0.001*"만든"
토픽 1: 0.001*"매장" + 0.001*"직접" + 0.001*"만든" + 0.001*"성수동에" + 0.001*"사람들이"
토픽 2: 0.001*"입장" + 0.001*"성수동에" + 0.001*"주년을" + 0.001*"침대" + 0.001*"창립"
토픽 3: 0.001*"만든" + 0.001*"생각보다" + 0.001*"com" + 0.001*"직접" + 0.001*"사람이"
토픽 4: 0.001*"오랜만에" + 0.001*"한번" + 0.001*"생각보다" + 0.001*"가서" + 0.001*"직접"

=== naver_blog_2021.json의 토픽 ===
토픽 0: 0.001*"생각보다" + 0.001*"전시" + 0.001*"제일" + 0.001*"한번" + 0.001*"보이는"
토픽 1: 0.001*"ㅋ" + 0.001*"아케인" + 0.001*"만든" + 0.001*"않고" + 0.001*"층에"
토픽 2: 0.001*"한번" + 0.001*"만든" + 0.001*"향수" + 0.001*"생각보다" + 0.001*"마음에"
토픽 3: 0.001*"만든" + 0.001*"전시" + 0.001*"예약" + 0.001*"직접" + 0.001*"생각보다"
토픽 4: 0.001*"FACTORY" + 0.001*"직접" + 0.001*"전시" + 0.001*"한번" + 0.001*"만나볼"

=== naver_blog_2022.json의 토픽 ===
토픽 0: 0.001*"직접" + 0.001*"만나볼" + 0.001*"콜라보" + 0.001*"저도" + 0.001*"이벤트"
토픽 1: 0.001*"직접" + 0.001*"어딕트" + 0.001*"예약" + 0.001*"생각보다" + 0.001*"한번"
토픽 2: 0.001*"com" + 0.001*"https" +

In [4]:
# 블로그 wordcloud
import re
import os
import json
from gensim import corpora, models
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib import font_manager, rc

# 기본 경로 설정
base_path = r"C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01"
stopwords_path = os.path.join(base_path, "stopwords_blog.txt")

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows의 경우
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 불용어 파일 읽기 함수
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stop_words = set(line.strip() for line in f)
        return stop_words
    except FileNotFoundError:
        print(f"불용어 파일이 존재하지 않습니다: {file_path}")
        return set()

# 텍스트 전처리 함수 (간단한 정규식 기반 토크나이저 사용)
def preprocess_text(text, stop_words):
    tokenizer = RegexpTokenizer(r'\w+')  # 단어 기준 토크나이저
    text = re.sub(r'\[.*?\]', '', text)  # 대괄호 내용 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    tokens = tokenizer.tokenize(text)  # 텍스트를 단어 단위로 토크나이징
    tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거
    return tokens

# 워드 클라우드 생성 함수
def generate_wordcloud(title, tokens):
    try:
        # 단어 빈도 계산
        freq = defaultdict(int)
        for word in tokens:
            freq[word] += 1
        if not freq:
            print(f"{title} 워드 클라우드에 사용할 단어가 없습니다.")
            return
        # 워드 클라우드 생성
        wordcloud = WordCloud(
            font_path=font_path,  # 한글 폰트 경로 설정
            background_color='white',
            width=800,
            height=600
        ).generate_from_frequencies(freq)
        # 워드 클라우드 시각화
        plt.figure(figsize=(10, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(title, fontsize=20)
        plt.tight_layout(pad=0)
        # 워드 클라우드를 이미지 파일로 저장
        save_path = os.path.join(base_path, f"{title}.png")
        plt.savefig(save_path)
        plt.close()
        print(f"{title} 워드 클라우드 저장 완료: {save_path}")
    except Exception as e:
        print(f"{title} 워드 클라우드 생성 중 오류 발생: {e}")

# 데이터 로드 및 합치기
def load_and_combine_data(file_names):
    combined_texts = []
    for file_name in file_names:
        file_path = os.path.join(base_path, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for year_data in data:
                    for entry in year_data.get(file_name.split('_')[2].split('.')[0], []):
                        content = entry.get('content', '')
                        combined_texts.append(content)
        except FileNotFoundError as fnf_error:
            print(f"파일을 찾을 수 없습니다: {fnf_error}")
        except Exception as e:
            print(f"{file_name} 처리 중 오류 발생: {e}")
    return combined_texts

# 2020-2021년 데이터 로드 및 합치기
file_names_2020_2021 = [
    "naver_blog_2020.json",
    "naver_blog_2021.json"
]
texts_2020_2021 = load_and_combine_data(file_names_2020_2021)

# 2022-2024년 데이터 로드 및 합치기
file_names_2022_2024 = [
    "naver_blog_2022.json",
    "naver_blog_2023.json",
    "naver_blog_2024.json"
]
texts_2022_2024 = load_and_combine_data(file_names_2022_2024)

# 전처리 및 워드 클라우드 생성
def process_and_generate_wordcloud(texts, title, stopwords_path):
    stop_words = load_stopwords(stopwords_path)
    all_tokens = []
    for text in texts:
        tokens = preprocess_text(text, stop_words)
        all_tokens.extend(tokens)
    generate_wordcloud(title, all_tokens)

# 2020-2021년 워드 클라우드 생성
process_and_generate_wordcloud(texts_2020_2021, "2020-2021년", stopwords_path)

# 2022-2024년 워드 클라우드 생성
process_and_generate_wordcloud(texts_2022_2024, "2022-2024년", stopwords_path)

2020-2021년 워드 클라우드 저장 완료: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\2020-2021년.png
2022-2024년 워드 클라우드 저장 완료: C:\Users\host\Desktop\PROJECT_SaSAC_01\PROJECT_SaSAC_01\2022-2024년.png
